In [3]:
#import libraries 
import numpy as np  
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
#from imblearn.over_sampling import RandomOverSampler 
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from adspy_shared_utilities.py import plot_two_class_knn

ModuleNotFoundError: No module named 'adspy_shared_utilities'

In [1197]:
#import data 
red_wine = pd.read_csv("wine+quality\winequality-red.csv", sep= ';', delimiter=None) 

#make a copy
rw = red_wine


In [1198]:
#remove all duplicates 
rw.drop_duplicates(inplace=True)


In [1199]:
#unify column names 
rw.rename(columns = {'fixed acidity':'fixed_acidity','volatile acidity':'volatile_acidity','citric acid':'citric_acid','residual sugar':'residual_sugar','free sulfur dioxide':'free_sulfur_dioxide', 'total sulfur dioxide':'total_sulfur_dioxide'}, inplace=True)

In [1200]:
#add a column 
#rw.insert(12, "colour", 'red')

In [1201]:
#add quality label column - low 1-4, medium 5-6, high 7-10 
rw['quality_label'] = rw['quality'].apply(lambda value: 'low'
if value <= 4 else 'medium'
if value <= 6 else 'high')
rw['quality_label'] = pd.Categorical(rw['quality_label'], 
categories=['low', 'medium', 'high']) 


In [1202]:
#check unique values of this column 
rw['quality_label'].unique()

['medium', 'high', 'low']
Categories (3, object): ['low', 'medium', 'high']

In [1203]:
#see data distribution 
rw['quality_label'].value_counts()

medium    1112
high       184
low         63
Name: quality_label, dtype: int64

Label Encoder 

In [1204]:
#make a copy 
rw1 = rw.copy()

In [1205]:
#first save label encoder 
lbl_enc = LabelEncoder()

#use function fit transform 
rw1['quality_label'] = lbl_enc.fit_transform(rw1['quality_label'])



In [1206]:
rw1['quality_label'].value_counts()
# low is 1 
# medium is 2 
# high is 0


#          low quality = 1-4 
#       medium quality = 5-6
#         high quality = 7-10 



2    1112
0     184
1      63
Name: quality_label, dtype: int64

Train, validation, test datasets 

In [1207]:
#split data
train, valid, test = np.split(rw1.sample(frac=1), [int(0.6*len(rw1)), int(0.8*len(rw1))])

In [1208]:
#scale data 
def scale_dataset(rw1, oversample=False):
    X = rw1[rw1.columns[:-1]].values #calling all columns until the last one 
    y = rw1[rw1.columns[-1]].values #calling only the last column 

    scaler = StandardScaler() # check rules here for group splitting 
    X = scaler.fit_transform(X) 

    data = np.hstack((X, np.reshape(y, (-1, 1))))
    # y is only 2D, so call numpy reshape to make it  3Dimensional Item 
    # because y is only 1 column

    return data, X, y

In [1209]:
#train data and run model 
train, X_train, y_train = scale_dataset(train)
valid, X_valid, y_valid = scale_dataset(valid)
test, X_test, y_test = scale_dataset(test)

knn_model = KNeighborsClassifier(n_neighbors=7) # dont try random state, define all parameters 
knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)


In [1210]:
#check classification report 
print(classification_report(y_test, y_pred)) 

# low is 1 
# medium is 2 
# high is 0

              precision    recall  f1-score   support

           0       0.93      0.74      0.83        35
           1       1.00      0.44      0.62         9
           2       0.94      0.99      0.97       228

    accuracy                           0.94       272
   macro avg       0.96      0.73      0.80       272
weighted avg       0.94      0.94      0.94       272



In [1211]:
# do a confusion matrix  
print(confusion_matrix(y_test, y_pred))
#google explaination

[[ 26   0   9]
 [  0   4   5]
 [  2   0 226]]


Add confusion matrix labels 

In [1212]:
#for label in rw.columns[:-1]:
 #   plt.hist(rw[rw["quality_label"]==0][label], color = 'blue', label = 'High Quality', alpha = 0.7, density=True)
  #  plt.hist(rw[rw["quality_label"]==1][label], color = 'red', label = 'Low Quality', alpha = 0.7, density=True)
   # plt.title(label)
    #plt.xlabel(label)
    #plt.legend()
    #plt.show()